In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../rings_fixed_radius"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
FIXED_RADIUS = 1.5

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 8
NUM_HIDDEN_LOCAL = 64
NUM_STATS = 16

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 300
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'baseline-mlp-%dsamples' % (SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

obs_rad = torch.ones(1) * FIXED_RADIUS
noise_sigma = torch.ones(1) * 0.05
if CUDA:
    obs_rad = obs_rad.cuda().to(DEVICE)
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (obs_rad, noise_sigma, N, K, D)

In [3]:
from local_enc import *
from global_oneshot import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
if CUDA:
    enc_z.cuda().to(DEVICE)
oneshot_eta = Oneshot_mu(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
if CUDA:
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_z)

In [4]:
from os_ep import *
train(models, EP, optimizer, Data, Model_Params, Train_Params)

epoch: 0\300 (13s),  eubo: -574259.451,  elbo: -683248.312,  ess: 1.000
epoch: 1\300 (12s),  eubo: -540529.708,  elbo: -629727.488,  ess: 1.000
epoch: 2\300 (12s),  eubo: -470686.329,  elbo: -583130.063,  ess: 1.000
epoch: 3\300 (12s),  eubo: -366132.606,  elbo: -515104.378,  ess: 1.000
epoch: 4\300 (13s),  eubo: -301749.546,  elbo: -460880.130,  ess: 1.000
epoch: 5\300 (12s),  eubo: -277432.896,  elbo: -436589.056,  ess: 1.000
epoch: 6\300 (12s),  eubo: -253024.658,  elbo: -404020.403,  ess: 1.000
epoch: 7\300 (13s),  eubo: -238112.624,  elbo: -370260.445,  ess: 1.000
epoch: 8\300 (12s),  eubo: -221224.857,  elbo: -331508.528,  ess: 1.000
epoch: 9\300 (15s),  eubo: -204577.673,  elbo: -303430.360,  ess: 1.000
epoch: 10\300 (15s),  eubo: -190382.779,  elbo: -290934.071,  ess: 1.000
epoch: 11\300 (15s),  eubo: -178348.096,  elbo: -281591.398,  ess: 1.000
epoch: 12\300 (15s),  eubo: -158473.150,  elbo: -270557.786,  ess: 1.000
epoch: 13\300 (15s),  eubo: -136055.517,  elbo: -254453.713, 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-d96b46a7abe2>", line 2, in <module>
    train(models, EP, optimizer, Data, Model_Params, Train_Params)
  File "../training.py", line 24, in train
    loss, metric_step, reused = objective(models, obs, SubTrain_Params)
  File "../objectives/os_ep.py", line 19, in EP
    q_z, p_z = enc_z.forward(obs, obs_mu, obs_rad, noise_sigma, N, K, sample_size, batch_size)
  File "../models/local_enc.py", line 26, in forward
    data_ck = torch.cat((obs, obs_mu[:, :, k, :].unsqueeze(-2).repeat(1,1,N,1), obs_rad.repeat(sample_size, batch_size, N, 1)), -1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, i

KeyboardInterrupt: 

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)

In [ ]:
BATCH_SIZE_TEST = 50
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
obs, metric_step, reused = test(models, EP, Data, Model_Params, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples(obs, q_mu, q_z, K, PATH)

In [ ]:
incremental_gap = symkls_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')